In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from indexer import clone_repo, build_index, query_index

# Clone the repository if not present
repo_url = 'https://github.com/viarotel-org/escrcpy'
repo_folder = '../data/escrcpy'
if not os.path.exists(repo_folder):
    clone_repo(repo_url, repo_folder)

In [3]:
from evaluate import load_evaluation_data, evaluate

In [4]:
eval_filepath = '../data/escrcpy-commits-generated.json'
data = load_evaluation_data(eval_filepath)

In [5]:
# eval_filepath = '../data/escrcpy-commits-generated.json'
# if os.path.exists(eval_filepath):
#     data = load_evaluation_data(eval_filepath)
#     evaluate(repo_folder, data)
# else:
#     print(f'No evaluation data file found at {eval_filepath}.')

In [16]:
extensions = [
    '1', 
    'CN', 
    'LICENSE', 
    'js', 
    'json', 
    'md', 
    'npmrc', 
    'nvmdrc', 
    'vue', 
    'yml',
]

chunk_to_file, embeddings, model = build_index(repo_folder, extensions)

Found 211 files.


No sentence-transformers model found with name microsoft/graphcodebert-base. Creating a new one with mean pooling.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Created 275 chunks.


In [18]:
total = len(data)
hit_count = 0
for item in data:
    query = item.get('question')
    expected_files = item.get('files', [])
    expected_files = set([os.path.join(repo_folder, file) for file in expected_files])
    
    ret_files = set(query_index(query, embeddings, model, chunk_to_file))
    
    if expected_files.intersection(ret_files):
        hit_count += 1
recall_at_10 = hit_count / total if total > 0 else 0

In [19]:
recall_at_10

0.17647058823529413

In [9]:
for item in data:
    query = item.get('question')
    expected_files = item.get('files', [])
    expected_files = set([os.path.join(repo_folder, file) for file in expected_files])
    
    ret_files = set(query_index(query, embeddings, model, chunk_to_file))

    break

In [10]:
expected_files, ret_files

({'../data/escrcpy/src/components/PreferenceForm/components/SelectDisplay/index.vue'},
 {'../data/escrcpy/PRIVACY.md',
  '../data/escrcpy/README.md',
  '../data/escrcpy/electron/resources/extra/linux/scrcpy/scrcpy.1',
  '../data/escrcpy/electron/resources/extra/mac-arm64/scrcpy/scrcpy.1',
  '../data/escrcpy/electron/resources/extra/mac-x64/scrcpy/scrcpy.1'})

In [15]:
extensions = set()
for item in data:
    expected_files = item.get('files', [])
    for file in expected_files:
        ext =file.split('.')[-1]

        if ext not in extensions:
            print(file, ext, item.get('question'))

        extensions.add(ext)

extensions

src/components/PreferenceForm/components/SelectDisplay/index.vue vue How does the SelectDisplay component handle the device options when retrieving display IDs?
electron/exposes/adb/helpers/index.js js How does the repository handle IPv6 addresses in ADB commands?
README-CN.md md Unable to detect device
src/locales/languages/zh-TW.json json What are the common user interface labels and messages implemented in Traditional Chinese?
.github/workflows/release-assets.yml yml What workflows support manual triggering in the repository?
.nvmdrc nvmdrc What is the purpose of the configuration specified in the version management file?
.npmrc.CN CN What configuration should domestic users apply to resolve dependency installation issues?
.npmrc npmrc How is the configuration for resolving dependency installation issues specified in the project?
LICENSE LICENSE What license terms and conditions govern the usage, reproduction, and distribution of the repository?
electron/resources/extra/linux/scrcpy

{'1', 'CN', 'LICENSE', 'js', 'json', 'md', 'npmrc', 'nvmdrc', 'vue', 'yml'}